In [1]:
import pandas as pd 
import os 
import fitz
import math

In [14]:
# 读入df_img
df_img = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/10k_distance.xlsx")

In [15]:
values_to_remove = ["华为2022.pdf", "万科2022.pdf", "601966.SH-玲珑轮胎-玲珑轮胎 山东玲珑轮胎股份有限公司2022年可持续发展报告英文版-2023-04-29.pdf", "002129.SZ-TCL中环-TCL中环 TCL中环新能源科技股份有限公司2022年度可持续发展报告(英文版)-2023-03-29.pdf", "000726.SZ-鲁泰A-鲁泰Ａ 鲁泰纺织可持续发展报告2022-英文版-2023-04-12.pdf"]
df_img = df_img[~df_img['PDF_name'].isin(values_to_remove)]

In [16]:
PDF_folder = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022"

In [17]:
df_PDF = pd.DataFrame()
df_PDF['PDF_name'] = df_img['PDF_name'].unique()

# 获取PDF页数

In [18]:
def pages(PDF_path):
    pdf_file = fitz.open(PDF_path)
    return pdf_file.page_count

In [19]:
# 获取PDF页数
df_PDF['PDF_pages'] = df_PDF['PDF_name'].apply(lambda x: pages(os.path.join(PDF_folder, x)))

# 获取PDF中图片数量

In [20]:
df_PDF = df_PDF.merge(df_img.groupby('PDF_name')['page'].count().reset_index(name= 'img_num'),
                     on='PDF_name', how='left')

# 获取图片大小（均值）

In [21]:
df_PDF = df_PDF.merge(df_img.groupby('PDF_name')['img_size'].mean().reset_index(name= 'img_size_avg'),
                     how='left', on='PDF_name')

In [22]:
# 计算图片数量与页数比
df_PDF['img_num_pages_ratio'] = df_PDF.apply(lambda row: row['img_num'] / row['PDF_pages'], axis=1)

# 计算图片大小和页数比
df_PDF['img_size_pages_ratio'] = df_PDF.apply(lambda row: row['img_size_avg'] / row['PDF_pages'], axis=1)

# 计算图文空间距离(平均） 与 页面对角线距离 比

In [23]:
def diagonal(PDF_path):
    pdf_file = fitz.open(PDF_path)
    width, height = pdf_file[2].rect.width, pdf_file[2].rect.height
    
    return math.sqrt(width**2 + height**2)

In [24]:
diagonal_series = df_PDF['PDF_name'].apply(lambda x: diagonal(os.path.join(PDF_folder, x)))
diagonal_series.index = df_PDF['PDF_name']

In [25]:
diagonal_compute = pd.merge(diagonal_series.reset_index(name = 'diag'), 
                           df_img.groupby('PDF_name')['distance'].mean().reset_index(name = 'dist_avg'),
                           on='PDF_name',
                           how = 'left')

In [26]:
diagonal_compute['dist_diag_ratio'] = diagonal_compute.apply(lambda row: row['dist_avg'] / row['diag'], axis=1)

In [28]:
df_PDF = pd.merge(df_PDF, diagonal_compute[['PDF_name', 'dist_diag_ratio']],
        on='PDF_name', how = 'left')
df_PDF.to_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/main_data.xlsx", index=False)

# 获取上市公司代码

In [ ]:
df_PDF = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/main_data.xlsx")

In [14]:
def corp_code(PDF_name):
    return str(PDF_name.split("-")[0][0:6]) if PDF_name.split("-")[0][-2:] != 'HK' else str(PDF_name.split("-")[0][0:5])

def corp_market(PDF_name):
    return PDF_name.split("-")[0][-2:]

In [15]:
df_PDF['Corp_code'] = df_PDF['PDF_name'].apply(corp_code)

In [10]:
df_PDF['Corp_market'] = df_PDF['PDF_name'].apply(corp_market)
# df_PDF = df_PDF[['Corp_code','Corp_market', 'PDF_name', 'PDF_pages', 'img_num', 'img_size_avg', 'img_num_pages_ratio',
#        'img_size_pages_ratio', 'dist_diag_ratio']]

In [20]:
df_PDF.to_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/main_data.xlsx", index=False)

In [19]:
df_PDF

,Corp_code,Corp_market,PDF_name,PDF_pages,img_num,img_size_avg,img_num_pages_ratio,img_size_pages_ratio,dist_diag_ratio
0,01138,HK,01138.HK-中远海能-中远海能 中远海能2022年可持续发展报告-2023-03-31...,81,32,77928.528832,0.395062,962.080603,0.287667
1,600872,SH,600872.SH-中炬高新-中炬高新 中炬高新2022年度社会责任暨可持续发展报告-202...,41,93,93096.083568,2.268293,2270.636185,0.483167
2,002916,SZ,002916.SZ-深南电路-深南电路 2022年可持续发展(暨ESG)报告-2023-03...,40,59,38031.556062,1.475000,950.788902,0.250278
3,00941,HK,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,50,68,77341.818769,1.360000,1546.836375,0.345360
4,002271,SZ,002271.SZ-东方雨虹-东方雨虹 2022年度可持续发展报告-2023-04-13.pdf,55,90,43160.205976,1.636364,784.731018,0.290214
...,...,...,...,...,...,...,...,...,...
70,600201,SH,600201.SH-生物股份-生物股份 金宇生物技术股份有限公司2022年年度可持续发展报告...,36,35,102463.497340,0.972222,2846.208259,0.344099
71,600588,SH,600588.SH-用友网络-用友网络 用友网络2022年度可持续发展(ESG)报告-202...,33,31,109075.470388,0.939394,3305.317284,0.333807
72,600989,SH,600989.SH-宝丰能源-宝丰能源 宁夏宝丰能源集团股份有限公司2022年可持续发展报告...,35,58,199669.598451,1.657143,5704.845670,0.333910
73,600019,SH,600019.SH-宝钢股份-宝钢股份 宝山钢铁股份有限公司2022年度可持续发展报告-20...,73,227,157117.564714,3.109589,2152.295407,0.320448


In [2]:
df_PDF = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/main_data.xlsx")


,Corp_code,Corp_market,PDF_name,PDF_pages,img_num,img_size_avg,img_num_pages_ratio,img_size_pages_ratio,dist_diag_ratio
0,1138,HK,01138.HK-中远海能-中远海能 中远海能2022年可持续发展报告-2023-03-31...,81,32,77928.528832,0.395062,962.080603,0.287667
1,600872,SH,600872.SH-中炬高新-中炬高新 中炬高新2022年度社会责任暨可持续发展报告-202...,41,93,93096.083568,2.268293,2270.636185,0.483167
2,2916,SZ,002916.SZ-深南电路-深南电路 2022年可持续发展(暨ESG)报告-2023-03...,40,59,38031.556062,1.475000,950.788902,0.250278
3,941,HK,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,50,68,77341.818769,1.360000,1546.836375,0.345360
4,2271,SZ,002271.SZ-东方雨虹-东方雨虹 2022年度可持续发展报告-2023-04-13.pdf,55,90,43160.205976,1.636364,784.731018,0.290214
...,...,...,...,...,...,...,...,...,...
70,600201,SH,600201.SH-生物股份-生物股份 金宇生物技术股份有限公司2022年年度可持续发展报告...,36,35,102463.497340,0.972222,2846.208259,0.344099
71,600588,SH,600588.SH-用友网络-用友网络 用友网络2022年度可持续发展(ESG)报告-202...,33,31,109075.470388,0.939394,3305.317284,0.333807
72,600989,SH,600989.SH-宝丰能源-宝丰能源 宁夏宝丰能源集团股份有限公司2022年可持续发展报告...,35,58,199669.598451,1.657143,5704.845670,0.333910
73,600019,SH,600019.SH-宝钢股份-宝钢股份 宝山钢铁股份有限公司2022年度可持续发展报告-20...,73,227,157117.564714,3.109589,2152.295407,0.320448


In [5]:
df_PDF['Pub_year'] = 2022
df_PDF['Pub_type'] = 'SUS'

df_PDF.to_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/main_data.xlsx", index=False)